In [4]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [5]:
SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly', 'https://www.googleapis.com/auth/classroom.coursework.students', 
          'https://www.googleapis.com/auth/classroom.courseworkmaterials', 'https://www.googleapis.com/auth/classroom.topics']

In [6]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('classroom', 'v1', credentials=creds)

In [35]:
# Call the Classroom API
results = service.courses().list(pageSize=10).execute()
courses = results.get('courses', [])

if not courses:
    print('No courses found.')

# Prints the names of the first 10 courses.
print('Courses:')
for course in courses:
    print(f"- {course['name']}")

Courses:
- Automation
- Wizard Data Analytics
- Vulcan Machine Learning
- Vulcan Data Visualization
- Vulcan Data Analytics
- Theia Machine Learning
- Vulcan Scholarship
- Ultron Data Analytics
- Ultron Data Visualization
- Ultron Machine Learning


In [37]:
course_input = input("Enter Course Name: ")
course_id = None

for course in courses:
    if course_input == course['name']:
        course_id = course['id']
        break

if course_id == None:
    print(f"ERROR: {course_input} course not found")
else:
    print(f'{course_input} found with ID {course_id}')

Automation found with ID 616952989203


In [8]:
topic_list = ['Capstone Project', 'Learning by Bulding', 
              'Quiz', 'Additional Assignment', 'Lecture Notes',
              'Material', 'Administration Deck']

In [9]:
try:
  for name in topic_list:
    topic = {
        "name": name
    }
    response = service.courses().topics().create(
      courseId=course_id,
      body=topic).execute()

    print('Topic created: {} - {}'.format(response['name'], response['topicId']))

except HttpError as error:
  print(f"An error occurred: {error}")

An error occurred: <HttpError 409 when requesting https://classroom.googleapis.com/v1/courses/616952989203/topics?alt=json returned "Requested entity already exists". Details: "Requested entity already exists">


In [10]:
topics = []

response = service.courses().topics().list(
    courseId=course_id).execute()
topics.extend(response.get('topic', []))

print('Topics : ')
for topic in topics:
    print('{} - {}'.format(topic['name'], topic['topicId']))

Topics : 
Administration Deck - 617018691211
Material - 617017450722
Lecture Notes - 617018677375
Additional Assignment - 617017420534
Quiz - 617018677370
Learning by Bulding - 617017468106
Capstone Project - 617016752483


In [9]:
coursework = {
    'title': '6: LBB: Neural Network & Deep Learning',
    'description': '''Dear Bapak/Ibu,

    Selamat atas keberhasilan-nya melewati 4 hari pembelajaran di kelas Neural Network dan Deep Learning! Dalam rangka meningkatkan kemampuan Bapak dan Ibu dalam mengimplementasikan ilmu yang didapatkan di kelas, kami menyediakan modul Learn by Building sebagai sarana Bapak dan Ibu untuk berlatih dalam membangun study case Neural Network dan Deep Learning.

    Berikut merupakan beberapa contoh Neural Network dan Deep Learning yang mungkin bisa menjadi acuan atau inspirasi Bapak/Ibu membuat sebuah study case:

    - https://rpubs.com/meilinie/lbbnn
    - (TABULAR) https://github.com/bagasbgy/keras-examples/tree/classification-dense
    - (IMAGE) https://github.com/bagasbgy/keras-examples/tree/image_classification-dense

    Kami sangat menyarankan bagi Bapak/Ibu yang berencana untuk ikut serta dalam Algoritma Career Support agar dapat mengerjakan modul tambahan ini sehingga dapat ditambahkan sebagai portofolio pribadi. Hal tersebut akan sangat membantu recruiter untuk memahami tingkat pengalaman case hands-on Bapak/Ibu.

    Berikut beberapa saran dataset yang bisa Bapak/Ibu gunakan sebagai case Neural Network dan Deep Learning analysis pada modul ini:

    - (TABULAR) IBM Attrition dataset https://github.com/bagasbgy/keras-examples/tree/classification-dense/data/data-clean.csv/ Sumber data didapat dari: https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset atau https://www.ibm.com/communities/analytics/watson-analytics-blog/hr-employee-attrition/
    - (IMAGE) MNIST dataset https://github.com/bagasbgy/keras-examples/tree/image_classification-dense/data/ Sumber data didapat dari https://www.kaggle.com/scolianni/mnistasjpg#img_10071.jpg
    - atau menggunakan data pilihan Bapak atau Ibu sendiri.

    Dalam pembuatan report, jangan lupa untuk meliputi hal-hal berikut:

    - Preprocess dataset, terutama part di mana data dipersiapkan agar dapat digunakan dalam framework keras
    - Pembuatan arsitektur neural network, dan penjelasan singkat terkait arsitektur yang digunakan
    - Persiapan model menggunakan "compile" dan fitting model
    - Evaluasi model seperti machine learning pada umumnya

    Hasil report analysis yang telah dikerjakan, silahkan diunggah ke dalam salah satu situs publikasi pilihan Bapak/Ibu (Github / Rpubs / Kaggle Kernel / Website pribadi), kemudian silahkan mencantumkan linknya saat melakukan Submit LBB. Link kepada situs publikasi pilihan Bapak/Ibu akan kami cantumkan pada CV Algoritma yang akan kami buatkan sebagai salah satu bagian dari Algoritma Career Support. Kami akan memberikan feedback terkait pekerjaan Bapak/Ibu yang mengumpulkan modul Learn by Building ini mengenai hal-hal yang dirasa bisa dikembangkan.

    Jika ada pertanyaan yang ingin Bapak/Ibu tanyakan, silahkan dikirim ke mentor@algorit.ma. Kami akan dengan senang hati membantu.

    Terima kasih dan good luck!''',
    'workType': 'ASSIGNMENT',
    'state': 'DRAFT',
    'topicId' : '617017495975'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()
print(f"Assignment created with ID {coursework.get('id')}")

Assignment created with ID 617018128546


# Administration Deck

In [41]:
from assets import Classwork

description = Classwork()

In [14]:
for topic in topics:
    if topic['name'] == 'Administration Deck':
        break 

topic

{'courseId': '616952989203',
 'topicId': '617018691211',
 'name': 'Administration Deck',
 'updateTime': '2023-08-01T04:33:47.411Z'}

In [22]:
coursework = {
    'title': 'Permohonan Sesi Mentoring',
    'description': description.administration_deck['Permohonan Sesi Mentoring'],
    'materials': [
        {'link': {'url': 'http://bit.ly/mentoring-onsite?authuser=0'}},
        {'link': {'url': 'https://github.com/teamalgoritma/community?authuser=0'}},
        {'link': {'url': 'https://drive.google.com/file/d/1SuU3e8hg9jfMO25kWang1MIaaePa2x-Y/view?usp=drive_web&authuser=0'}},
    ],
    'topicId' : topic['topicId'],
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material Permohonan Sesi Mentoring created


In [23]:
coursework = {
    'title': 'How to Ask Questions',
    'description': description.administration_deck['How to Ask Questions'],
    'materials': [
        {'link': {'url': 'https://github.com/teamalgoritma/community?authuser=0'}},
    ],
    'topicId' : topic['topicId'],
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material How to Ask Questions created


In [24]:
coursework = {
    'title': 'Administration Deck',
    'description': description.administration_deck['Administration Deck'],
    'materials': [
        {'link': {'url': 'https://drive.google.com/file/d/1ivjryti41IekS55HSj43bC8TXLaZZrii/view?usp=drive_web&authuser=0'}},
        {'link': {'url': 'https://drive.google.com/file/d/1vUG9v_Aie-aHW4U93FD7LKcjfxilHG5u/view?usp=drive_web&authuser=0'}}
    ],
    'topicId' : topic['topicId'],
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material Administration Deck created


# Material

In [25]:
for topic in topics:
    if topic['name'] == 'Material':
        break 

topic

{'courseId': '616952989203',
 'topicId': '617017450722',
 'name': 'Material',
 'updateTime': '2023-08-01T04:33:45.732Z'}

In [26]:
coursework = {
    'title': '4. M : Interactive Plotting (IP)',
    'materials': [
        {'link': {'url': 'https://bit.ly/ip_material?authuser=0'}},
    ],
    'topicId' : topic['topicId'],
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 4. M : Interactive Plotting (IP) created


In [27]:
coursework = {
    'title': '3. M : Data Visualization (DV)',
    'materials': [
        {'link': {'url': 'https://bit.ly/dviz_material?authuser=0'}},
    ],
    'topicId' : topic['topicId'],
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 3. M : Data Visualization (DV) created


In [28]:
coursework = {
    'title': '2. M : Practical Statistics (PS)',
    'materials': [
        {'link': {'url': 'https://bit.ly/ps_material?authuser=0'}},
    ],
    'topicId' : topic['topicId'],
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 2. M : Practical Statistics (PS) created


In [29]:
coursework = {
    'title': '1. M : Programming for Data Science (P4DS)',
    'materials': [
        {'link': {'url': 'https://bit.ly/p4ds_material?authuser=0'}},
    ],
    'topicId' : topic['topicId'],
    'state' : 'DRAFT'
}
coursework = service.courses().courseWorkMaterials().create(
    courseId=course_id, body=coursework).execute()

print(f"Material {coursework.get('title')} created")

Material 1. M : Programming for Data Science (P4DS) created


# Quiz

In [38]:
for topic in topics:
    if topic['name'] == 'Quiz':
        break 

topic

{'courseId': '616952989203',
 'topicId': '617018677370',
 'name': 'Quiz',
 'updateTime': '2023-08-01T04:33:40.769Z'}

In [42]:
coursework = {
    'title': '3. Q: Interactive Plotting (IP)',
    'description': description.quiz['3. Q: Interactive Plotting (IP)'],
    'materials': [
        {'link': {'url': 'https://algoritmaonline.com/quizzes/interactive-plotting-quiz/?authuser=0'}}
    ],
    'topicId' : topic['topicId'],
    'workType': 'ASSIGNMENT',
    'maxPoints': 1,
    'state' : 'DRAFT'
}
coursework = service.courses().courseWork().create(
    courseId=course_id, body=coursework).execute()

print(f"Assignment {coursework.get('title')} created")

AttributeError: 'Classwork' object has no attribute 'quiz'